In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn_pandas import DataFrameMapper


price_pre = pd.read_csv('train.csv')

k=10

########构建热力图，分析相关系数###########
corrmatrix = price_pre.corr()

cols = corrmatrix.nlargest(k,'SalePrice')['SalePrice'].index

cm1 = price_pre[cols].corr()

hm2 = sns.heatmap(cm1,square=True,annot=True,cmap='RdPu',fmt='.2f',annot_kws={'size':10})
###################################

cols1 = ['OverallQual', 'GrLivArea', 'GarageCars','TotalBsmtSF', 'FullBath', 'TotRmsAbvGrd', 'YearBuilt']

total = price_pre.isnull().sum().sort_values(ascending=False)
percent = (price_pre.isnull().sum()/price_pre.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total,percent],axis=1,keys = ['Total','Percent'])

data1 = price_pre.drop(missing_data[missing_data['Total']>1].index,axis=1)

data2 = data1.drop(data1.loc[data1['Electrical'].isnull()].index)

data2.isnull().sum().max()

feature_data = data2.drop(['SalePrice'],axis=1)
target_data = data2['SalePrice']
feature_data_selected = feature_data[cols1]
svc = svm.SVC(gamma=0.001)
parameters = {'kernel': ['linear'], 'C': [1.0, 10.0, 100.0, 1000.0]}
searchCV = GridSearchCV(estimator=svc, param_grid=parameters, cv=5, verbose=3, return_train_score=True)

###### onehot编码######
mapper_arr = []
for col in cols1:
    mapper_arr.append(([col], OneHotEncoder(sparse=False)))
onehot_dfm = DataFrameMapper(mapper_arr, df_out=True, default=None)

onehot_dfm.fit(feature_data[cols1])
tmp_df = onehot_dfm.transform(feature_data[cols1])
X_train = pd.concat([tmp_df, feature_data_selected.drop(cols1, axis=1)], axis=1)
######################


train_X, val_X, train_y, val_y = train_test_split(X_train, target_data, random_state = 100)

searchCV.fit(train_X,train_y)


test_data = pd.read_csv('test.csv')
test_input = test_data[['OverallQual', 'GrLivArea', 'GarageCars','TotalBsmtSF', 'FullBath', 'TotRmsAbvGrd', 'YearBuilt']]

############查找并填充NaN值##############
test_input[test_input.isnull().values==True]
X_in = test_input.fillna(0)
#######################################

mapper_arr = []
for col in cols1:
    mapper_arr.append(([col], OneHotEncoder(sparse=False)))
onehot_dfm = DataFrameMapper(mapper_arr, df_out=True, default=None)

onehot_dfm.fit(X_in[cols1])
tmp_df = onehot_dfm.transform(X_in[cols1])
test_X = pd.concat([tmp_df, X_in.drop(cols1, axis=1)], axis=1)


val_predictions = searchCV.predict(test_X)
submission = pd.DataFrame({'Id': test.Id, 'SalePrice': val_predictions})
submission.to_csv('submission.csv', index=False)









In [ ]:
#一些pandas应用

#将空值换为none
data_new = price_pre.applymap(lambda x: None if x == '' else x)

data_new.dropna(axis=1, how='all', inplace=True) #删除全部为空值的列

# 计算每列空值比例
df_null = pd.DataFrame(data=data_new.dtypes, columns=['col_type'], index=data_new.dtypes.index)
df_null.loc[:,'null_percent'] = data_new.isnull().sum() * 100 / data_new.shape[0]

# 删除大于一定比例的空值字段
df_null['delete'] = np.where(df_null['null_percent'] > 80, 1, 0)
drop_cols = list(df_null[df_null.delete == 1].index)
data_new = data_new.drop(labels=drop_cols, axis=1)




In [ ]:
########两个pandas合并###########
price_pre = pd.read_csv('train.csv')
test_price = pd.read_csv('test.csv')
dd = pd.merge(price_pre,test_price,how = 'outer')

In [ ]:
########ordinalEncoder&onehotEncoder########
#from sklearn.preprocessing import OrdinalEncoder
#enc_Ordinal = preprocessing.OrdinalEncoder()
X = [['male', 'from US', 'uses Safari'],['female', 'from Europe', 'uses Firefox']]
#enc_Ordinal.fit(X)

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np
X = np.array(['US','Africa','EU'])
enc_onehot =LabelEncoder()
enc_onehot.fit(X)
new_features = enc_onehot.transform(X)
new_features = new_features.reshape(-1, 1)
ohe = preprocessing.OneHotEncoder(sparse=False)
ohe.fit_transform(new_features)

#genders = ['female', 'male']
#locations = ['from Africa', 'from Asia', 'from Europe', 'from US']
#browsers = ['uses Chrome', 'uses Firefox', 'uses IE', 'uses Safari']
#enc = preprocessing.OneHotEncoder(categories=[genders, locations, browsers])
# Note that for there are missing categorical values for the 2nd and 3rd
# feature
#X = [['male', 'from US', 'uses Safari'], ['female', 'from Europe', 'uses Firefox']]
#enc.fit(X) 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import re
from sklearn.linear_model import *
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics import mean_squared_error,mean_absolute_error


data = pd.read_csv('data.csv')

positions = ['CAM', 'CB', 'CDM', 'CF', 'CM', 'LAM',
       'LB', 'LCB', 'LCM', 'LDM', 'LF', 'LM', 'LS', 'LW', 'LWB', 'RAM', 'RB', 'RCB', 'RCM', 'RDM', 'RF',
       'RM', 'RS', 'RW', 'RWB']

#plt.figure(figsize=(50,40))
#sns.heatmap(data.corr(), annot=True)

pos_field = {'Def': re.findall(r"\wB|\w\wB",str([x for x in positions])),
            'Mid': re.findall(r"\wM|\w\wM",str([x for x in positions])),
            'Att': list(set(re.findall(r"\wS|\wF|\wW",str([x for x in positions]))))}

data2 = data.drop(columns=['Unnamed: 0','Name','Photo', 'Nationality', 'Flag','Club', 'Club Logo', 'Release Clause',
                           'Jersey Number','Value','Real Face','Loaned From','Body Type', 'LS', 'ST', 'RS', 'LW',
       'LF', 'CF', 'RF', 'RW', 'LAM', 'CAM', 'RAM', 'LM', 'LCM', 'CM', 'RCM',
       'RM', 'LWB', 'LDM', 'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB'])

########将数据格式转换为float###########
def reshape_value(value):
    new_value = value.replace(',', '').replace('€', '').replace('K','')
    return np.float(new_value)
####################################
data2['Wage']=data2['Wage'].apply(reshape_value)

#plt.figure(figsize=(50,40))
#corrmatrix = data2.corr()
#cols = corrmatrix.nlargest(10,'Wage')['Wage'].index
#sns.heatmap(data2[cols].corr(), annot=True)


cols1 = ['Overall', 'International Reputation', 'Reactions', 'Potential', 'Composure', 'Special', 'Vision', 'ShortPassing']
labels = data2['Wage']
X = data2[cols1]
X_in = X.fillna(0)

###########归一化#############
mapper_arr = []
for col in cols1:
    model = MinMaxScaler()
    mapper_arr.append(([col], model))
minmax_dfm = DataFrameMapper(mapper_arr, default=None, df_out=True)
minmax_dfm.fit(X_in)
tmp_df = minmax_dfm.transform(X_in)
df = pd.concat([tmp_df, X_in.drop(cols1, axis=1)], axis=1)
#############################

##########Label编码###########
def labelencoder(X_in):
    mapper_arr = []
    for col in cols1:
        le = LabelEncoder()
        mapper_arr.append(([col], le))
    label_dfm = DataFrameMapper(mapper_arr, default=None, df_out=True)
    label_dfm.fit(X_in)
    tmp_df = label_dfm.transform(X_in)
    df1 = pd.concat([tmp_df, X_in.drop(cols1, axis=1)], axis=1)
    return df1
#############################
def onehot(X_in):
    mapper_arr = []
    for col in cols1:
        mapper_arr.append(([col], OneHotEncoder(sparse=False)))
    onehot_dfm = DataFrameMapper(mapper_arr, df_out=True, default=None)
    onehot_dfm.fit(X_in)
    tmp_df = onehot_dfm.transform(X_in)
    df2 = pd.concat([tmp_df, X_in.drop(cols1, axis=1)], axis=1)
    return df2



train_X, val_X, train_y, val_y = train_test_split(df, labels, random_state = 100)
svr_lin = SVR(kernel='rbf', C=1e3, gamma=0.1)
svr_lin.fit(train_X,train_y)
pred = svr_lin.predict(val_X)
mean_squared_error(val_y, pred)



#总结回归和分类区别，输出不同。定量输出称为回归，或者说是连续变量预测；定性输出称为分类，或者说是离散变量预测。
#举个例子：预测明天的气温是多少度，这是一个回归任务；预测明天是阴、晴还是雨，就是一个分类任务。
#logistic回归不是回归是分类。

In [ ]:
import matplotlib.pyplot as plt

########一些分析数据需要的图的生成########
var = 'GrLivArea'
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)
data.plot.scatter(x=var, y='SalePrice', ylim=(0,800000));

k = 10 #number of variables for heatmap
cols = corrmat.nlargest(k, 'SalePrice')['SalePrice'].index
cm = np.corrcoef(df_train[cols].values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)

sns.set()
cols = ['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt']
sns.pairplot(df_train[cols], size = 2.5)
plt.show()